  # **AKADEMI EDUCATION**
# **Première cohorte (2025): Science des données et intelligence artificielle**
#### *Phase 5: PROJET DE SCIENCE DES DONNÉES*

**Noms des étudiants du groupe: Riché FLEURINORD et Micka LOUIS**   
**Rythme d’apprentissage: Autonome**  
**Date de soutenance: 27 octobre 2025**  
**Noms des instructeurs: Wedter JEROME et Geovany Batista Polo LAGUERRE**  
**Lien de l’article de blog (lien du dépôt GitHub): https://github.com/richefleuriord/Bank_Customer_Churn_Prediction.git**

# *3-Préparation des données*

## Préparation Complète des Données

La **préparation des données** constitue une étape essentielle du projet, garantissant la qualité, la cohérence et la pertinence du jeu de données avant la modélisation.  
Cette section détaille le processus complet appliqué au dataset de churn bancaire afin d’obtenir une base exploitable pour les modèles prédictifs.

---

### Objectif général

Transformer le jeu de données brut en un ensemble propre, cohérent et prêt pour la modélisation, tout en assurant la reproductibilité et la compatibilité avec le futur déploiement de l’application Streamlit.

---

### Étapes principales

1. **Chargement et vérifications initiales**  
   - Import du jeu de données depuis le répertoire `../Data/Customer-Churn-Records.csv`.  
   - Vérification du nombre de lignes, colonnes, doublons et valeurs manquantes.  
   - Analyse de la distribution de la variable cible `Exited` pour évaluer le déséquilibre de classes.

2. **Nettoyage et Feature Engineering**  
   - Suppression des colonnes non pertinentes: `CustomerId`, `RowNumber`, `Surname`, `Complain`.  
   - Création de nouvelles variables dérivées:
     - `BalanceToSalaryRatio` : rapport entre le solde et le salaire estimé.  
     - `AgeCategory` : segmentation des clients en catégories — *Jeune*, *Moyen* et *Âgé*.  
     - `NumOfProductsBinary` : codage binaire du nombre de produits détenus (0 si un seul, 1 si plusieurs).  
     - `ActiveCredit` : interaction entre la carte de crédit et l’activité du client.  
     - `LowSatisfaction` : indicateur des clients insatisfaits (score ≤ 2).  

3. **Encodage des variables catégorielles**  
   - Application d’un **OrdinalEncoder** sur les variables ordinales: `Card Type`, `AgeCategory`.  
   - Application d’un **OneHotEncoder** sur les variables nominales: `Gender`, `Geography` (avec suppression d’une modalité pour éviter la colinéarité).  
   - Les variables numériques sont conservées sans transformation.

4. **Séparation du jeu de données**  
   - Division en trois sous-ensembles:  
     - **Train (70%)** : apprentissage du modèle.  
     - **Validation (15%)** : ajustement des hyperparamètres.  
     - **Test (15%)** : évaluation finale.  
   - Utilisation du paramètre `stratify=y` pour conserver la proportion de classes dans chaque sous-ensemble.

5. **Transformation et assemblage final**  
   - Application du `ColumnTransformer` pour combiner les encodages.  
   - Récupération des noms de colonnes issus des transformations pour garantir la compatibilité des modèles.  
   - Construction de `DataFrames` propres et alignés : `X_train`, `X_val`, `X_test`.

6. **Sauvegarde et traçabilité**  
   - Enregistrement des fichiers préparés (`X_train_prepared.csv`, `y_train_prepared.csv`, etc.).  
   - Sauvegarde des colonnes finales encodées dans `columns_final.pkl` afin d’assurer la reproductibilité et le déploiement cohérent du pipeline.

---

### Résumé des points clés

| Étape | Objectif | Outils utilisés |
|:------|:----------|:----------------|
| Chargement | Importation et vérification du dataset | `pandas` |
| Nettoyage | Suppression des colonnes inutiles, création de ratios | `numpy`, `pandas` |
| Feature Engineering | Ajout de variables explicatives pertinentes | `pandas` |
| Encodage | Transformation catégorielle (ordinal + one-hot) | `sklearn.preprocessing` |
| Split des données | Constitution des ensembles train/val/test | `train_test_split` |
| Sauvegarde | Export des objets pour modélisation et déploiement | `joblib`, `csv` |

---

*Cette préparation rigoureuse garantit la qualité du pipeline analytique, renforce la robustesse des modèles et facilite l’intégration fluide dans la phase de modélisation et de déploiement Streamlit.*


In [4]:
# ============================================================
# PRÉPARATION COMPLÈTE DES DONNÉES (CORRIGÉE ET COMPLÈTE)
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
import joblib
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# CHARGEMENT DES DONNÉES
# ============================================================
df = pd.read_csv("../Data/Customer-Churn-Records.csv")
print(f"Chargement réussi : {df.shape[0]} lignes, {df.shape[1]} colonnes")

# ============================================================
# VÉRIFICATIONS DE BASE
# ============================================================
print("\n--- Vérifications de base ---")
print("Valeurs manquantes :", df.isnull().sum().sum())
print("Doublons :", df.duplicated().sum())
print("Distribution Exited :\n", df['Exited'].value_counts(normalize=True) * 100)

# ============================================================
# FEATURE ENGINEERING
# ============================================================

# Suppression des colonnes non pertinentes
df = df.drop(columns=["CustomerId", "RowNumber", "Surname", "Complain"], errors='ignore')

# Ratio Balance / Salaire
df['BalanceToSalaryRatio'] = df['Balance'] / (df['EstimatedSalary'] + 1)

# Catégorisation âge
df['AgeCategory'] = pd.cut(
    df['Age'], bins=[17, 35, 55, 100],
    labels=['Jeune', 'Moyen', 'Âgé']
)
df = df.drop(columns=['Age'], errors='ignore')

# Codage binaire du nombre de produits : 0 si 1 produit, 1 si 2 ou plus
df['NumOfProductsBinary'] = np.where(df['NumOfProducts'] == 1, 0, 1)
df = df.drop(columns=['NumOfProducts'], errors='ignore')

# Variables combinées et binaires
df['ActiveCredit'] = df['IsActiveMember'] * df['HasCrCard']
df['LowSatisfaction'] = (df['Satisfaction Score'] <= 2).astype(int)

# ============================================================
# ENCODAGE DES VARIABLES CATÉGORIELLES
# ============================================================

# Variables ordinales et nominales
ordinal_cols = ['Card Type', 'AgeCategory']
nominal_cols = ['Gender', 'Geography']

# ColumnTransformer pour encodage
preprocessor = ColumnTransformer(transformers=[
    ('ord', OrdinalEncoder(), ordinal_cols),
    ('nom', OneHotEncoder(drop='first', sparse=False), nominal_cols)
], remainder='passthrough')  # les colonnes numériques restent intactes

# ============================================================
# SÉPARATION FEATURES / TARGET
# ============================================================
X = df.drop('Exited', axis=1)
y = df['Exited']

# ============================================================
# SPLIT TRAIN / VAL / TEST
# ============================================================
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# ============================================================
# TRANSFORMATION DES COLONNES CATÉGORIELLES
# ============================================================

# Transformation
X_train_transformed = preprocessor.fit_transform(X_train)
X_val_transformed = preprocessor.transform(X_val)
X_test_transformed = preprocessor.transform(X_test)

# Récupération des noms des colonnes (compatible toutes versions)
ord_names = ordinal_cols
try:
    nom_names = preprocessor.named_transformers_['nom'].get_feature_names_out(nominal_cols)
except AttributeError:
    nom_names = preprocessor.named_transformers_['nom'].get_feature_names(nominal_cols)

num_names = [col for col in X_train.columns if col not in ordinal_cols + nominal_cols]
all_columns = list(ord_names) + list(nom_names) + list(num_names)

# Construction des DataFrames finaux
X_train = pd.DataFrame(X_train_transformed, columns=all_columns)
X_val = pd.DataFrame(X_val_transformed, columns=all_columns)
X_test = pd.DataFrame(X_test_transformed, columns=all_columns)

# ============================================================
# SAUVEGARDE DES COLONNES FINALES POUR LE DÉPLOIEMENT
# ============================================================
joblib.dump(list(X_train.columns), r"C:\Users\HP\course\phase_5\Models\columns_final.pkl")

# ============================================================
# SAUVEGARDE DES DONNÉES PRÉPARÉES
# ============================================================
X_train.to_csv("../Data/X_train_prepared.csv", index=False)
y_train.to_csv("../Data/y_train_prepared.csv", index=False)
X_val.to_csv("../Data/X_val_prepared.csv", index=False)
y_val.to_csv("../Data/y_val_prepared.csv", index=False)
X_test.to_csv("../Data/X_test_prepared.csv", index=False)
y_test.to_csv("../Data/y_test_prepared.csv", index=False)

print("Préparation complète des données terminée")


Chargement réussi : 10000 lignes, 18 colonnes

--- Vérifications de base ---
Valeurs manquantes : 0
Doublons : 0
Distribution Exited :
 Exited
0    79.62
1    20.38
Name: proportion, dtype: float64
Préparation complète des données terminée
